In [2]:
import pandas as pd
import requests
import json
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup, Tag
from datetime import date
import time

In [3]:
today = date.today()
formatted_date = today.strftime("%Y-%m-%d")

input_filename = f'sheets/list_with_applyLink_{formatted_date}.xlsx'

df = pd.read_excel(input_filename)

In [4]:
def extract_main_content(html_source):
    """
    BeautifulSoup을 사용하여 HTML의 header와 footer 사이의 내용만 추출합니다.
    """
    original_soup = BeautifulSoup(html_source, "html.parser")

    header = original_soup.find("header")
    footer = original_soup.find("footer")

    # header와 footer 사이의 컨텐츠만 담을 리스트
    content_tags = []

    if header:
        for element in header.next_siblings:
            if element is footer:
                break
            # Tag 객체인 경우에만 추가 (NavigableString 같은 다른 타입 제외)
            if isinstance(element, Tag):
                content_tags.append(element)
    else:
        # header가 없으면 body 전체를 대상으로 하되, footer는 제거
        body_content = original_soup.find_all("body")
        if body_content:
            for element in body_content:
                if element is not footer:
                    content_tags.append(element)

    # 추출된 내용으로 새로운 soup 객체를 만들어 순수한 텍스트만 반환
    new_soup = BeautifulSoup("<body></body>", "html.parser")
    body = new_soup.body
    for tag in content_tags:
        body.append(tag)

    # 최종적으로 정제된 텍스트 반환
    return new_soup.get_text(separator=" ", strip=True)

In [5]:
API_URL = "http://localhost:11434/v1/chat/completions"
MODEL_NAME = "gpt-oss"

def call_local_llm(html_content):
    """
    로컬 LLM에 정제된 텍스트를 보내 자격요건과 우대사항을 추출합니다.
    """
    headers = {"Content-Type": "application/json"}

    prompt = f"""
    다음은 채용 공고 페이지의 내용입니다. 이 내용에서 '자격요건'과 '우대사항'을 찾아서 각각 정리하라.
    결과는 반드시 아래와 같은 JSON 형식으로만 응답하라. 만약 내용이 없다면 빈 리스트([])로 응답하라.

    {{
      "자격요건": [
        "자격요건1",
        "자격요건2",
        ....
      ],
      "우대사항": [
        "우대사항1",
        "우대사항2",
        ....
      ]
    }}

    --- 공고 내용 시작 ---
    {html_content}
    --- 공고 내용 끝 ---
    """

    data = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.0,
    }

    try:
        response = requests.post(
            API_URL, headers=headers, data=json.dumps(data), timeout=300
        )
        response.raise_for_status()
        response_json = response.json()
        content_str = response_json["choices"][0]["message"]["content"]

        # JSON 블록을 추출하기 위한 정규 표현식
        # ```json ... ``` 또는 단순히 {...} 형태를 찾습니다.
        json_match = re.search(r"```json\s*(\{.*?\})\s*```", content_str, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            # ```json ... ``` 형태가 아니면 전체 내용을 JSON으로 시도
            json_str = content_str.strip()

        parsed_content = json.loads(json_str)
        return parsed_content
    except requests.exceptions.RequestException as e:
        print(f"  -> LLM API 호출 오류: {e}")
        return None
    except (json.JSONDecodeError, KeyError, IndexError) as e:
        print(f"  -> LLM 응답 파싱 오류: {e}")
        print(f"  -> 받은 내용: {content_str[:300]}")  # 받은 내용 일부 출력
        return None

In [ ]:
def main(df):
    # 1. 셀레니움 웹 드라이버 설정
    driver = webdriver.Chrome()
    results = []

    # 2. 각 링크를 순회하며 정보 추출
    for index, row in df.iterrows():

        apply_link = row["지원 링크"]
        print(f"처리 중 ({index + 1}/{len(df)}): {apply_link}")

        job_data = {
            "지원 링크": apply_link,  # 실제 링크 변수 사용
            "자격요건": "추출 실패",
            "우대사항": "추출 실패",
        }

        try:
            driver.get(apply_link)
            WebDriverWait(driver, 10).until(
                lambda d: d.execute_script("return document.readyState === 'complete'")
            )

            apply_page_html = driver.page_source

            # 메인 컨텐츠만 추출
            main_text = extract_main_content(apply_page_html)

            if not main_text.strip():
                print("  -> 처리할 텍스트 내용이 없습니다.")
                results.append(job_data)
                continue

            # LLM을 호출하여 정보 추출
            print(".....")
            extracted_info = call_local_llm(main_text)

            if extracted_info:
                job_data["자격요건"] = "\n".join(extracted_info.get("자격요건", []))
                job_data["우대사항"] = "\n".join(extracted_info.get("우대사항", []))
                print("  -> 정보 추출 완료.")
            else:
                print("  -> 정보 추출 실패.")

        except Exception as e:
            print(f"  -> 처리 중 오류 발생: {type(e).__name__} - {e}")
            # 오류 발생 시 다음 루프로 넘어감

        results.append(job_data)

    driver.quit()

    # 3. 결과 저장
    if results:
        result_df = pd.DataFrame(results)
        # "지원 링크"를 기준으로 원본과 결과 병합
        final_df = pd.merge(df, result_df, on="지원 링크", how="left")

        print(f"\n작업 완료!")
        return final_df
    else:
        print("\n처리된 결과가 없어 파일을 저장하지 않았습니다.")

In [7]:
start_time = time.time()  # 시작 시간 기록

result_df = main(df)

# output_filename = f'sheets/ai_jobs_final_results_{formatted_date}.xlsx'
# result_df.to_excel(output_filename, index=False)

end_time = time.time()  # 종료 시간 기록
elapsed = end_time - start_time
# print(f"\n최종 결과가 '{output_filename}' 파일에 저장되었습니다.")
print(f"\n최종 결과 산출 완료.")
print(f"총 소요 시간: {elapsed:.1f}초 ({elapsed/60:.1f}분)")

처리 중 (1/113): https://jobs.careers.microsoft.com/global/en/job/1856383
.....
  -> 정보 추출 완료.
처리 중 (2/113): https://recruit.navercloudcorp.com/rcrt/view.do?annoId=30003735&lang=ko
.....
  -> 정보 추출 완료.
처리 중 (3/113): https://www.lgresearch.ai/careers/view?seq=250
.....
  -> 정보 추출 완료.
처리 중 (4/113): https://career.doosan.com/dsp/sa/RecList.jsp?REC_ID=1000353083&mode=goDetail
.....
  -> 정보 추출 완료.
처리 중 (5/113): https://recruit.navercloudcorp.com/rcrt/view.do?annoId=30003836&lang=ko
.....
  -> 정보 추출 완료.
처리 중 (6/113): https://www.lgresearch.ai/careers/view?seq=253
.....
  -> 정보 추출 완료.
처리 중 (7/113): https://career.doosan.com/dsp/sa/RecList.jsp?REC_ID=1000353021&mode=goDetail
.....
  -> 정보 추출 완료.
처리 중 (8/113): https://recruit.cj.net/recruit/ko/recruit/recruit/bestDetail.fo?zz_jo_num=J20250826034037
.....
  -> 정보 추출 완료.
처리 중 (9/113): https://careers.upstage.ai//o/ai-agent-engineer
.....
  -> 정보 추출 완료.
처리 중 (10/113): https://careers.upstage.ai/o/ai-agent-engineer
.....
  -> 정보 추출 완료.
처리 중 (11/113): 

In [8]:
output_filename = f'sheets/ai_jobs_final_results_{formatted_date}.xlsx'
result_df.to_excel(output_filename, index=False)

In [14]:
# 자격요건 또는 우대사항이 null이거나 '추출 실패'인 행만 추출
fail_df = result_df[
    result_df["자격요건"].isnull()
    | result_df["우대사항"].isnull()
    | (result_df["자격요건"] == "추출 실패")
    | (result_df["우대사항"] == "추출 실패")
    | (result_df["자격요건"] == "")
    | (result_df["우대사항"] == "")
]

In [17]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회사      133 non-null    object
 1   제목      133 non-null    object
 2   경력      133 non-null    object
 3   근무형태    133 non-null    object
 4   학력      133 non-null    object
 5   근무지역    133 non-null    object
 6   링크      133 non-null    object
 7   지원 링크   133 non-null    object
 8   자격요건    133 non-null    object
 9   우대사항    133 non-null    object
dtypes: object(10)
memory usage: 10.5+ KB


In [ ]:
fail_df

<class 'pandas.core.frame.DataFrame'>
Index: 66 entries, 0 to 132
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   회사      66 non-null     object
 1   제목      66 non-null     object
 2   경력      66 non-null     object
 3   근무형태    66 non-null     object
 4   학력      66 non-null     object
 5   근무지역    66 non-null     object
 6   링크      66 non-null     object
 7   지원 링크   66 non-null     object
 8   자격요건    66 non-null     object
 9   우대사항    66 non-null     object
dtypes: object(10)
memory usage: 5.7+ KB


In [16]:
start_time = time.time()  # 시작 시간 기록

max_attempts = 10
attempt = 1

while attempt < max_attempts:
    # 자격요건 또는 우대사항이 null이거나 '추출 실패'인 행만 추출
    fail_df = result_df[
        result_df["자격요건"].isnull()
        | result_df["우대사항"].isnull()
        | (result_df["자격요건"] == "추출 실패")
        | (result_df["우대사항"] == "추출 실패")
        | (result_df["자격요건"] == "")
        | (result_df["우대사항"] == "")
    ]

    # 모두 정상적으로 추출되었으면 반복 종료
    if fail_df.empty:
        break

    print(f"\n=== {attempt+1}번째 재시도: {len(fail_df)}건 ===")
    # 실패한 행만 main 함수에 재전달
    if "자격요건" in fail_df.columns and "우대사항" in fail_df.columns:
        fail_df = fail_df.drop(columns=["자격요건", "우대사항"])
    retry_result = main(fail_df)

    # 기존 result_df에서 실패한 부분만 retry_result로 업데이트
    for idx, row in retry_result.iterrows():
        mask = result_df["지원 링크"] == row["지원 링크"]
        result_df.loc[mask, ["자격요건", "우대사항"]] = row[
            ["자격요건", "우대사항"]
        ].values

    attempt += 1
    
print('종료')


=== 2번째 재시도: 66건 ===
처리 중 (1/66): https://jobs.careers.microsoft.com/global/en/job/1856383
.....
  -> 정보 추출 완료.
처리 중 (18/66): https://toss.im/career/job-detail?job_id=4100792003
.....
  -> 정보 추출 완료.
처리 중 (19/66): https://medtronic.wd1.myworkdayjobs.com/en-US/MedtronicCareers/job/Seoul-Seoul-Korea/Korea-Channel-Data-Analyst--Contract-_R37500?locationCountry=7a5a2aadf9d34086a2bfbfd408bc28da
  -> 처리할 텍스트 내용이 없습니다.
처리 중 (20/66): https://tydtr0dj.ninehire.site/job_posting/CG6BujWw
.....
  -> 정보 추출 완료.
처리 중 (22/66): https://kakaopay.career.greetinghr.com/ko/o/158068
.....
  -> 정보 추출 완료.
처리 중 (23/66): https://kakaopay.career.greetinghr.com/ko/o/158068
.....
  -> 정보 추출 완료.
처리 중 (24/66): https://kakaopay.career.greetinghr.com/ko/o/158068
.....
  -> 정보 추출 완료.
처리 중 (25/66): https://kakaopay.career.greetinghr.com/ko/o/158068
.....
  -> 정보 추출 완료.
처리 중 (26/66): https://kakaopay.career.greetinghr.com/ko/o/158068
.....
  -> 정보 추출 완료.
처리 중 (27/66): https://kakaopay.career.greetinghr.com/ko/o/158068
..

KeyboardInterrupt: 

In [ ]:
start_time = time.time()  # 시작 시간 기록

max_attempts = 10
attempt = 1

while attempt < max_attempts:
    # 자격요건 또는 우대사항이 null이거나 '추출 실패'인 행만 추출
    fail_df = result_df[
        result_df["자격요건"].isnull()
        | result_df["우대사항"].isnull()
        | (result_df["자격요건"] == "추출 실패")
        | (result_df["우대사항"] == "추출 실패")
        | (result_df["자격요건"] == "")
        | (result_df["우대사항"] == "")
    ]

    # 모두 정상적으로 추출되었으면 반복 종료
    if fail_df.empty:
        break

    print(f"\n=== {attempt+1}번째 재시도: {len(fail_df)}건 ===")
    # 실패한 행만 main 함수에 재전달
    if "자격요건" in fail_df.columns and "우대사항" in fail_df.columns:
        fail_df = fail_df.drop(columns=["자격요건", "우대사항"])
    retry_result = main(fail_df)

    # 기존 result_df에서 실패한 부분만 retry_result로 업데이트
    for idx, row in retry_result.iterrows():
        mask = result_df["지원 링크"] == row["지원 링크"]
        result_df.loc[mask, ["자격요건", "우대사항"]] = row[
            ["자격요건", "우대사항"]
        ].values

    attempt += 1

# 최종 결과 분리
fail_df = result_df[
    result_df["자격요건"].isnull()
    | result_df["우대사항"].isnull()
    | (result_df["자격요건"] == "추출 실패")
    | (result_df["우대사항"] == "추출 실패")
]
success_df = result_df[
    result_df["자격요건"].notnull()
    & result_df["우대사항"].notnull()
    & (result_df["자격요건"] != "추출 실패")
    & (result_df["우대사항"] != "추출 실패")
]

output_filename = f"sheets/ai_jobs_final_results_{formatted_date}.xlsx"
result_df.to_excel(output_filename, index=False)

end_time = time.time()  # 종료 시간 기록
elapsed = end_time - start_time
print(f"\n최종 결과가 '{output_filename}' 파일에 저장되었습니다.")
print(f"총 소요 시간: {elapsed:.1f}초 ({elapsed/60:.1f}분)")

처리 중 (1/113): https://jobs.careers.microsoft.com/global/en/job/1856383
  -> LLM 호출하여 정보 추출 중...
  -> LLM 전체 응답: {
  "id": "chatcmpl-898",
  "object": "chat.completion",
  "created": 1756727161,
  "model": "gpt-oss",
  "system_fingerprint": "fp_ollama",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"자격요건\": [],\n  \"우대사항\": []\n}",
        "reasoning": "The user says: \"다음은 채용 공고 페이지의 내용입니다. 이 내용에서 '자격요건'과 '우대사항'을 찾아서 각각 정리하라. 결과는 반드시 아래와 같은 JSON 형식으로만 응답하라. 만약 내용이 없다면 빈 리스트([])로 응답하라.\"\n\nThen they provide a placeholder: \"--- 공고 내용 시작 --- Loading... --- 공고 내용 끝 ---\"\n\nIt seems the actual content is not provided; it's just \"Loading...\". So there is no content. According to instruction, if content is missing, we should return empty lists for both.\n\nThus output:\n\n{\n  \"자격요건\": [],\n  \"우대사항\": []\n}\n\nWe must respond only in JSON format. No other text."
      },
      "finish_reason": "stop"
    }
  ],
  "usage

KeyError: "None of [Index(['자격요건', '우대사항'], dtype='object')] are in the [index]"

In [16]:
fail_df.columns

Index(['회사', '제목', '경력', '근무형태', '학력', '근무지역', '링크', '지원 링크', '자격요건', '우대사항'], dtype='object')